## Preparations

You must have executed the cells in `training-demo.ipynb` and started an inference server with Apolo Deploy to continue.

## Installing Dependencies

In [ ]:
!pip install apolo-sdk apolo-extras

## Testing the Inference Server

In [ ]:
import requests
import json
import apolo_sdk

# Get the Triton server URL from the Apolo SDK
async with apolo_sdk.get() as client:
    apolo_token = await client.config.token()
    async with client.jobs.list(name=["triton"], statuses=[apolo_sdk.JobStatus.RUNNING]) as job_iter:
        jobs = [job async for job in job_iter]
        server_url = jobs[0].http_url

# Define the endpoint URL
url = f"{server_url}/v2/models/onnx-iris-perceptron-production/versions/1/infer"

# Create the payload with normalized input values
payload = {
    "id": "test_request",
    "inputs": [
        {
            "name": "X",
            "shape": [1, 4],
            "datatype": "FP32",
            "data": [0.22, 0.63, 0.07, 0.04]
        }
    ]
}

# Set the headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {apolo_token}"
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Print the response
print("Status Code:", response.status_code)
print("Response:")
print(response.text)
print(json.dumps(response.json(), indent=2))